In [1]:
import numpy as np

# Data

In [2]:
sigma = 0
gamma = 0.5
S, C = [], []

In [3]:
with open(r"E:\datasets\dac\sellers.txt", "r") as f:
    lines = f.readlines()
    for l in lines:
        S.extend(l.split())

with open(r"E:\datasets\dac\customers.txt", "r") as f:
    lines = f.readlines()
    for l in lines:
        C.extend(l.split())

In [4]:
alpha = np.zeros(len(S))
alpha_bar = np.zeros(len(S))
A = np.zeros((len(S), len(C)))
types = ["Blue", "Red"]
p = np.zeros((len(S), len(types)))
S_id2idx = {id: idx for idx, id in enumerate(S)}
C_id2idx = {id: idx for idx, id in enumerate(C)}
t_id2idx = {id: idx for idx, id in enumerate(types)}

def extract_alpha(line):
    s1, s2 = line.split()
    return s1[1:-1], float(s2)

def extract_ap(line):
    s1, s2 = line.split()
    s1 = s1[1:-1]
    id1, id2 = s1.split(",")
    return id1, id2, float(s2)

with open(r"E:\datasets\dac\alpha.txt", "r") as f:
    lines = f.readlines()
    for l in lines:
        id, val = extract_alpha(l)
        alpha[S_id2idx[id]] = val

with open(r"E:\datasets\dac\alpha_bar.txt", "r") as f:
    lines = f.readlines()
    for l in lines:
        id, val = extract_alpha(l)
        alpha_bar[S_id2idx[id]] = val

with open(r"E:\datasets\dac\a.txt", "r") as f:
    lines = f.readlines()
    for l in lines:
        id1, id2, val = extract_ap(l)
        A[S_id2idx[id1]][C_id2idx[id2]] = val

with open(r"E:\datasets\dac\p.txt", "r") as f:
    lines = f.readlines()
    for l in lines:
        id, color, val = extract_ap(l)
        p[S_id2idx[id]][t_id2idx[color[1:-1]]] = val

In [5]:
beta = np.maximum(alpha, alpha_bar)

In [6]:
K = np.sum(A.T @ (gamma + (1 - gamma) * p[:,0]))
# equal to (up to 4th decimal):
# np.sum([[A[s][c] * (gamma + (1-gamma) * p[s,0]) 
#         for c in range(len(C))] for s in range(len(S))])
K

7304938.168550002

In [7]:
k = A.T @ ((1 - gamma) * (p[:,1] - p[:,0]))

In [ ]:
# p_diff = (p[:,1] - p[:,0])
# k = [([A[s][c] * (1 - gamma) * p_diff
#              for s in range(len(S))]) for c in range(len(C))]
# k

# Gurobi

In [8]:
import gurobipy as gp
from gurobipy import GRB

In [9]:
to_del = np.where(beta > 1)[0]

In [10]:
A2 = np.delete(A, to_del, axis=0)
beta2 = np.delete(beta, to_del, axis=0)

In [11]:
rhs = beta * A.sum(axis=-1)
# equal to:
# rhs = [beta[s] * np.sum(A[s,:]) for s in range(len(S))]

In [ ]:
rhs = beta2 * A2.sum(axis=-1)
rhs.shape

In [12]:
m = gp.Model()
x = m.addMVar(len(C), vtype=GRB.BINARY, name="x")
m.setObjective(K + k @ x, GRB.MAXIMIZE)
m.addConstr(A @ x <= rhs)
m.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-16
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-9700K CPU @ 3.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7000 rows, 2881 columns and 61693 nonzeros
Model fingerprint: 0x76d7b759
Variable types: 0 continuous, 2881 integer (2881 binary)
Coefficient statistics:
  Matrix range     [8e-01, 1e+05]
  Objective range  [3e-17, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+00, 2e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 7306261.8988
Presolve removed 7000 rows and 2881 columns
Presolve time: 0.05s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.13 seconds (0.01 work units)
Thread count was 1 (of 8 available proces

In [13]:
opt_gurobi = m.getObjective().getValue()

x = []
for v in m.getVars():
    x.append(v.x)
x = np.array(x)
assert (A @ x <= rhs).all()
x

array([0., 0., 0., ..., 0., 0., 0.])

In [15]:
x_cplex = []

with open(r"E:\datasets\dac\best_cplex.txt", "r") as f:
    lines = f.readlines()
    for l in lines:
        ss = l.split()
        vals = [int(val) for val in ss if int(val) <= 1]
        x_cplex.extend(vals)

x_cplex = np.array(x_cplex)
# assert (A @ x_cplex <= rhs).all()
len(x_cplex)

AssertionError: 

In [19]:
len(np.where(A @ x_cplex > rhs)[0])

1029

In [ ]:
assert np.array_equal(x, x_cplex)

In [16]:
sum(x), sum(x_cplex)

(92.0, 92)